# Quick Import For Embedding Matplotlib Animations Into Your Notebook
@author Jake Vanderplas
@url http://jakevdp.github.io/blog/2013/05/12/embedding-matplotlib-animations/
@date May 12, 2013
@editedBy Chris Fortuna

I made some changes so it would work in my environment (Windows 10, python 3.5). If you have compatibility issues check out the original website, the comments to the original post helped me get it running, they might help you out as well.


# Imports
You'll call display_animation(anim) passing in your animation created with MatPlotLib. You'll want to do this at the end of a cell for it to display properly

In [2]:
from tempfile import NamedTemporaryFile
import matplotlib.pyplot as plt
import base64
from matplotlib import animation
from IPython.display import HTML


VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        anim.save("test.mp4", fps=20, extra_args=['-vcodec', 'libx264'])

        video = open("test.mp4","rb").read()

    anim._encoded_video = base64.b64encode(video).decode('utf-8')
    # prevent figure displayed as a PNG below the animation
    plt.close()
    return VIDEO_TAG.format(anim._encoded_video)

#  this is an automatic hook which is nice if you want to display as soon as you call the animation but causes
#  problems if it's in an object
# animation.Animation._repr_html_ = anim_to_html


def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

# Some Example Animations

In [3]:
from matplotlib import animation
import numpy as np
%matplotlib inline


# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-2, 2))
line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

# call our new function to display the animation
display_animation(anim)

In [3]:
"""
A simple example of an animated plot... In 3D!
"""
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

def Gen_RandLine(length, dims=2) :
    """
    Create a line using a random walk algorithm

    length is the number of points for the line.
    dims is the number of dimensions the line has.
    """
    lineData = np.empty((dims, length))
    lineData[:, 0] = np.random.rand(dims)
    for index in range(1, length) :
        # scaling the random numbers by 0.1 so
        # movement is small compared to position.
        # subtraction by 0.5 is to change the range to [-0.5, 0.5]
        # to allow a line to move backwards.
        step = ((np.random.rand(dims) - 0.5) * 0.1)
        lineData[:, index] = lineData[:, index-1] + step

    return lineData

def update_lines(num, dataLines, lines) :
    for line, data in zip(lines, dataLines) :
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2,:num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
ax = p3.Axes3D(fig)

# Fifty lines of random 3-D lines
data = [Gen_RandLine(25, 3) for index in range(50)]

# Creating fifty line objects.
# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]

# Setting the axes properties
ax.set_xlim3d([0.0, 1.0])
ax.set_xlabel('X')

ax.set_ylim3d([0.0, 1.0])
ax.set_ylabel('Y')

ax.set_zlim3d([0.0, 1.0])
ax.set_zlabel('Z')

ax.set_title('3D Test')

def show_animation():
    # Creating the Animation object
    anim = animation.FuncAnimation(fig, update_lines, 25, fargs=(data, lines),
                                  interval=50, blit=False)
    return display_animation(anim)

print("hi hi hi ")  #  try moving print to the end of the cell, it won't work, display animation has the return to the cell end

show_animation()

hi hi hi 
